## Key Concepts in this Notebook¶

- pipe
- factory
- EntityRuler
- PhraseMatcher
- Matcher

### Einführung in Spacys EntityRuler¶

Die Python-Bibliothek spaCy bietet verschiedene Methoden zur Durchführung von regelbasiertem NER. Eine solche Methode ist der EntityRuler.

Der EntityRuler ist eine SpaCy-Fabrik, die es ermöglicht, eine Reihe von Mustern mit entsprechenden Beschriftungen zu erstellen. Eine Factory in spaCy ist eine Reihe von in spaCy vorinstallierten Klassen und Funktionen, die festgelegte Aufgaben ausführen. Im Fall des EntityRuler ermöglicht die vorliegende Factory dem Benutzer, einen EntityRuler zu erstellen, ihm eine Reihe von Anweisungen zu geben und diese Anweisungen dann zum Suchen und Beschriften von Entitäten zu verwenden.

Sobald der Benutzer den EntityRuler erstellt und ihm eine Reihe von Anweisungen gegeben hat, kann er ihn als neue Pipe zur spaCy-Pipeline hinzufügen. Ich habe in früheren Notizbüchern kurz über Pfeifen gesprochen, aber vielleicht ist es sinnvoll, hier ausführlicher darauf einzugehen.

Ein Rohr ist ein Bestandteil einer Rohrleitung. Der Zweck einer Pipeline besteht darin, Eingabedaten zu erfassen, bestimmte Operationen an diesen Eingabedaten auszuführen und diese Operationen dann entweder als neue Daten oder als extrahierte Metadaten auszugeben. Ein Rohr ist ein einzelner Bestandteil einer Rohrleitung. Im Fall von spaCy gibt es einige verschiedene Pipes, die unterschiedliche Aufgaben erfüllen. Der Tokenizer tokenisiert den Text in einzelne Token; Der Parser analysiert den Text, und der NER identifiziert Entitäten und beschriftet sie entsprechend. Alle diese Daten werden im Doc-Objekt gespeichert, wie wir in Notebook 01_01 dieser Serie gesehen haben.

Es ist wichtig, sich daran zu erinnern, dass Pipelines sequentiell sind. Dies bedeutet, dass Komponenten früher in einer Pipeline Einfluss darauf haben, was spätere Komponenten erhalten. Manchmal ist diese Reihenfolge wichtig, was bedeutet, dass spätere Rohre von früheren Rohren abhängen. In anderen Fällen ist diese Reihenfolge nicht unbedingt erforderlich, was bedeutet, dass spätere Pipes ohne frühere Pipes funktionieren können. Es ist wichtig, dies im Hinterkopf zu behalten, wenn Sie benutzerdefinierte spaCy-Modelle (oder jede andere Pipeline) erstellen.

In diesem Notizbuch werden wir uns den EntityRuler als Komponente der Pipeline eines SpaCy-Modells genauer ansehen. Standardmäßige spaCy-Modelle sind mit einem NER-Modell vorinstalliert; Sie werden jedoch nicht mit einem EntityRuler geliefert. Um einen EntityRuler in ein spaCy-Modell zu integrieren, muss er als neue Pipe erstellt, mit Anweisungen versehen und dann dem Modell hinzugefügt werden. Sobald dies abgeschlossen ist, kann der Benutzer das neue Modell mit dem EntityRuler auf der Festplatte speichern.

Die vollständige Dokumentation von spaCy EntityRuler finden Sie hier: https://spacy.io/api/entityruler.

Dieses Notizbuch fasst diese Dokumentation für Laien zusammen und bietet einige Beispiele dafür in Aktion.

### Demonstration von EntityRuler in Aktion¶

Im folgenden Code werden wir eine neue Pipe in das handelsübliche kleine englische Modell von spaCy einführen. Der Zweck dieses EntityRulers besteht darin, kleine Dörfer in Polen korrekt zu identifizieren.

In [1]:
#import the requsite library
import spacy

#Buold upon the Spacy Small Model
nlp = spacy.load("en_core_web_sm")

#Sample text
text = "The village of Treblinka is in Poland. Treblinka was also an extermination camp."

#Create the Doc object
doc = nlp(text)

#extract entities
for ent in doc.ents:
  print(ent.text,ent.label_)

Treblinka GPE
Poland GPE


Abhängig von der von Ihnen verwendeten Modellversion können einige Ergebnisse variieren.

Die Ausgabe des obigen Codes zeigt die kleinen Modelle von spaCy zur Identifizierung von Treblinka, einem kleinen Dorf in Polen. Wie aus dem Beispieltext hervorgeht, war es im Zweiten Weltkrieg auch ein Vernichtungslager. Im ersten Satz markierte das spaCy-Modell Treblinka als LOC (Standort) und im zweiten Satz wurde es völlig übersehen. Beides ist entweder ungenau oder falsch. Ich hätte ORG für den zweiten Satz akzeptiert, da das Modell von spaCy nicht weiß, wie ein Vernichtungslager zu klassifizieren ist, aber diese Ergebnisse zeigen, dass das Modell nicht in der Lage ist, Daten zu verallgemeinern. Der Grund? Es gibt ein paar, aber ich vermute, dass dem Model das Wort Treblinka nie begegnet ist.

Dies ist ein häufiges Problem im NLP für bestimmte Domänen. Oftmals scheitern die Domänen, in denen wir Modelle einsetzen möchten, von der Stange, weil sie nicht auf domänenspezifische Texte trainiert wurden. Wir können dieses Problem jedoch entweder über den EntityRuler von spaCy oder durch das Training eines neuen Modells lösen. Wie wir in den nächsten Notizbüchern sehen werden, können wir mit dem EntityRuler von spaCy beides problemlos erreichen.

Lassen Sie uns das Problem zunächst beheben, indem wir dem Modell Anweisungen zur korrekten Identifizierung von Treblinka geben. Der Einfachheit halber verwenden wir das GPE-Label von spaCy. In einem späteren Notizbuch werden wir ein Modell vermitteln, um Treblinka im letztgenannten Kontext korrekt als Konzentrationslager zu identifizieren.

In [2]:
#Import the requisite library
import spacy

#Build upon the spaCy Small Model
nlp = spacy.load("en_core_web_sm")

#Sample text
text = "The village of Treblinka is in Poland. Treblinka was also an extermination camp."

#Create the EntityRuler
ruler = nlp.add_pipe("entity_ruler")

#List of Entities and Patterns
patterns = [
                {"label": "GPE", "pattern": "Treblinka"}
            ]

ruler.add_patterns(patterns)


doc = nlp(text)

#extract entities
for ent in doc.ents:
    print (ent.text, ent.label_)

Treblinka GPE
Poland GPE
Treblinka GPE


Wenn Sie den obigen Code ausgeführt haben und festgestellt haben, dass Sie die gleiche Ausgabe hatten, dann haben Sie alles richtig gemacht. Diese Methode ist fehlgeschlagen. Warum? Die Antwort liegt im Konzept der Pipelines. Wir haben den EntityRuler erstellt und zur Pipeline des spaCy-Modells hinzugefügt, aber standardmäßig fügt spaCy am Ende der Pipeline eine neue Pipe hinzu. Um die Pipeline zu visualisieren, verwenden wir analysierte_pipes() von spaCy.

In [3]:
nlp.analyze_pipes()

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False},
  'entity_ruler': {'assigns': ['doc.ents', 'token.ent_type', 'token.ent_iob'],
   'requires': [],
   'scores': ['ents_f', 'ent

Dies kann zunächst etwas schwierig zu lesen sein, aber es zeigt uns die Reihenfolge, in der unsere Pipes eingerichtet sind, und ein paar andere wichtige Informationen zu jeder Pipe. Wenn wir „ner“ finden, bemerken wir, dass „entity_ruler“ dahinter steht.

Damit unser EntityRuler Vorrang hat, müssen wir ihn nach der „ner“-Pipe zuweisen, wie das folgende Beispiel in dieser Zeile zeigt:

__Ruler__ = __nlp.add_pipe(“entity_ruler”, after=“ner”)__

In [4]:
#Build upon the spaCy Small Model
nlp = spacy.load("en_core_web_sm")

#Sample text
text = "The village of Treblinka is in Poland. Treblinka was also an extermination camp."

#Create the EntityRuler
ruler = nlp.add_pipe("entity_ruler", after="ner")

#List of Entities and Patterns
patterns = [
                {"label": "GPE", "pattern": "Treblinka"}
            ]

ruler.add_patterns(patterns)


doc = nlp(text)

#extract entities
for ent in doc.ents:
    print (ent.text, ent.label_)

Treblinka GPE
Poland GPE
Treblinka GPE


Beachten Sie jetzt, dass unser EntityRuler vor der „ner“-Pipe funktioniert und daher Entitäten vorab findet und sie beschriftet, bevor die NER sie erreicht. Da es früher in der Pipeline kommt, haben seine Metadaten Vorrang vor der späteren „ner“-Pipe.

### Einführung komplexer Regeln und Varianz in den EntityRuler (Erweitert)¶


In manchen Fällen können Beschriftungen eine festgelegte Art von Varianz aufweisen, die einem oder mehreren bestimmten Mustern folgt. Ein solches Beispiel (in der spaCy-Dokumentation enthalten) sind Telefonnummern. In den Vereinigten Staaten gibt es verschiedene Formen von Telefonnummern. Die standardmäßige formale Methode ist (xxx)-xxx-xxxx, es ist jedoch nicht ungewöhnlich, xxx-xxx-xxxx oder xxxxxxxxxx zu sehen. Wenn der Inhaber der Telefonnummer dieselbe Nummer an jemanden außerhalb der USA weitergibt, dann +1(xxx)-xxx-xxxx.

Wenn Sie in einer US-amerikanischen Domäne arbeiten, können Sie RegEx-Formeln an den Mustervergleicher übergeben, um alle diese Instanzen zu erfassen.

Der spaCy EntityRuler ermöglicht es dem Benutzer auch, eine Vielzahl komplexer Regeln und Varianzen einzuführen (unter anderem über RegEx), indem er die Regeln an das Muster übergibt. Es gibt viele Argumente, die man den Mustern anführen kann. Eine vollständige Liste finden Sie unter: https://spacy.io/usage/rule-based-matching. Um herauszufinden, wie diese funktionieren, empfehle ich die Verwendung der spaCy Matcher-Demo: https://explosion.ai/demos/matcher.

Im folgenden Beispiel arbeiten wir mit einem Beispiel aus der spaCy-Dokumentation, in dem wir eine Telefonnummer aus einem Text extrahieren. Dieselbe Aufgabe kann auch über RegEx erledigt werden.

In [5]:
#Import the requisite library
import spacy

#Sample text
text = "This is a sample number (555) 555-5555."

#Build upon the spaCy Small Model
nlp = spacy.blank("en")

#Create the Ruler and Add it
ruler = nlp.add_pipe("entity_ruler")

#List of Entities and Patterns (source: https://spacy.io/usage/rule-based-matching)
patterns = [
                {"label": "PHONE_NUMBER", "pattern": [{"ORTH": "("}, {"SHAPE": "ddd"}, {"ORTH": ")"}, {"SHAPE": "ddd"},
                {"ORTH": "-", "OP": "?"}, {"SHAPE": "dddd"}]}
            ]
#add patterns to ruler
ruler.add_patterns(patterns)



#create the doc
doc = nlp(text)

#extract entities
for ent in doc.ents:
    print (ent.text, ent.label_)

(555) 555-5555 PHONE_NUMBER


## How to use the spaCy Matcher

### Basic Example

In [6]:
import spacy
from spacy.matcher import Matcher

In [9]:
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
pattern = [{"LIKE_EMAIL": True}]
matcher.add("EMAIL_ADDRESS", [pattern])
doc = nlp("This is an email address: wmattingly@aol.com")
matches = matcher(doc)

In [10]:
print(matches)

[(16571425990740197027, 6, 7)]


In [11]:
print(nlp.vocab[matches[0][0]].text)

EMAIL_ADDRESS


### Von Matcher übernommene Attribute

- ORTH - The exact verbatim of a token (str)
- TEXT - The exact verbatim of a token (str)
- LOWER - The lowercase form of the token text (str)
- LENGTH - The length of the token text (int)
- IS_ALPHA
- IS_ASCII
- IS_DIGIT
- IS_LOWER
- IS_UPPER
- IS_TITLE
- IS_PUNCT
- IS_SPACE
- IS_STOP
- IS_SENT_START
- LIKE_NUM
- LIKE_URL
- LIKE_EMAIL
- SPACY
- POS
- TAG
- MORPH
- DEP
- LEMMA
- SHAPE
- ENT_TYPE
- - Custom extension attributes (Dict[str, Any])
- OP

 ### Grabbing all Proper Nouns

In [14]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN"}]
matcher.add("PROPER_NOUNS", [pattern])
doc = nlp(text)
matches = matcher(doc)
print (len(matches))
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])

0


In [15]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}]
matcher.add("PROPER_NOUNS", [pattern])
doc = nlp(text)
matches = matcher(doc)
print (len(matches))
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])


0


In [16]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}]
matcher.add("PROPER_NOUNS", [pattern], greedy='LONGEST')
doc = nlp(text)
matches = matcher(doc)
print (len(matches))
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])

0


In [18]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}, {"POS": "VERB"}]
matcher.add("PROPER_NOUNS", [pattern], greedy='LONGEST')
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1])
print (len(matches))
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])

0


In [19]:
import json
with open ("data/alice.json", "r") as f:
    data = json.load(f)

FileNotFoundError: ignored

In [20]:
speak_lemmas = ["think", "say"]
text = data[0][2][0].replace( "`", "'")
matcher = Matcher(nlp.vocab)
pattern1 = [{'ORTH': "'"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': "'"}, {"POS": "VERB", "LEMMA": {"IN": speak_lemmas}}, {"POS": "PROPN", "OP": "+"}, {'ORTH': "'"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': "'"}]
pattern2 = [{'ORTH': "'"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': "'"}, {"POS": "VERB", "LEMMA": {"IN": speak_lemmas}}, {"POS": "PROPN", "OP": "+"}]
pattern3 = [{"POS": "PROPN", "OP": "+"},{"POS": "VERB", "LEMMA": {"IN": speak_lemmas}}, {'ORTH': "'"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': "'"}]
matcher.add("PROPER_NOUNS", [pattern1, pattern2, pattern3], greedy='LONGEST')
for text in data[0][2]:
    text = text.replace("`", "'")
    doc = nlp(text)
    matches = matcher(doc)
    matches.sort(key = lambda x: x[1])
    print (len(matches))
    for match in matches[:10]:
        print (match, doc[match[1]:match[2]])

NameError: ignored

## How to Use RegEx in Python


In [21]:
import re

In [22]:
pattern = r"((\d){1,2} (January|February|March|April|May|June|July|August|September|October|November|December))"

text = "This is a date 2 February. Another date would be 14 August."
matches = re.findall(pattern, text)
print (matches)

[('2 February', '2', 'February'), ('14 August', '4', 'August')]


In [23]:
text = "This is a date February 2. Another date would be 14 August."
matches = re.findall(pattern, text)
print (matches)

[('14 August', '4', 'August')]


In [24]:
pattern = r"(((\d){1,2}( (January|February|March|April|May|June|July|August|September|October|November|December)))|(((January|February|March|April|May|June|July|August|September|October|November|December) )(\d){1,2}))"

text = "This is a date February 2. Another date would be 14 August."
matches = re.findall(pattern, text)
print (matches)


[('February 2', '', '', '', '', 'February 2', 'February ', 'February', '2'), ('14 August', '14 August', '4', ' August', 'August', '', '', '', '')]


In [25]:
text = "This is a date February 2. Another date would be 14 August."
iter_matches = re.finditer(pattern, text)
print (iter_matches)

In [26]:
text = "This is a date February 2. Another date would be 14 August."
iter_matches = re.finditer(pattern, text)
print (iter_matches)
for hit in iter_matches:
    print (hit)

<re.Match object; span=(15, 25), match='February 2'>
<re.Match object; span=(49, 58), match='14 August'>


In [27]:
text = "This is a date February 2. Another date would be 14 August."
iter_matches = re.finditer(pattern, text)
for hit in iter_matches:
    start = hit.start()
    end = hit.end()
    print (text[start:end])


February 2
14 August


## How to Use RegEx in spaCy

In [28]:
#Import the requisite library
import spacy

#Sample text
text = "This is a sample number 555-5555."

#Build upon the spaCy Small Model
nlp = spacy.blank("en")

#Create the Ruler and Add it
ruler = nlp.add_pipe("entity_ruler")

#List of Entities and Patterns (source: https://spacy.io/usage/rule-based-matching)
patterns = [
                {"label": "PHONE_NUMBER", "pattern": [{"SHAPE": "ddd"},
                {"ORTH": "-", "OP": "?"}, {"SHAPE": "dddd"}]}
            ]
#add patterns to ruler
ruler.add_patterns(patterns)

#create the doc
doc = nlp(text)

#extract entities
for ent in doc.ents:
    print (ent.text, ent.label_)

555-5555 PHONE_NUMBER


In [29]:
pattern = r"((\d){3}-(\d){4})"
text = "This is a sample number 555-5555."
matches = re.findall(pattern, text)
print (matches)

[('555-5555', '5', '5')]


In [30]:
#Import the requisite library
import spacy

#Sample text
text = "This is a sample number (555) 555-5555."

#Build upon the spaCy Small Model
nlp = spacy.blank("en")

#Create the Ruler and Add it
ruler = nlp.add_pipe("entity_ruler")

#List of Entities and Patterns (source: https://spacy.io/usage/rule-based-matching)
patterns = [
                {
                    "label": "PHONE_NUMBER", "pattern": [{"TEXT": {"REGEX": "((\d){3}-(\d){4})"}}
                                                        ]
                }
            ]
#add patterns to ruler
ruler.add_patterns(patterns)


#create the doc
doc = nlp(text)

#extract entities
for ent in doc.ents:
    print (ent.text, ent.label_)

In [31]:
#Import the requisite library
import spacy

#Sample text
text = "This is a sample number (555) 555-5555."

#Build upon the spaCy Small Model
nlp = spacy.blank("en")

#Create the Ruler and Add it
ruler = nlp.add_pipe("entity_ruler")

#List of Entities and Patterns (source: https://spacy.io/usage/rule-based-matching)
patterns = [
                {
                    "label": "PHONE_NUMBER", "pattern": [{"TEXT": {"REGEX": "((\d){3}-(\d){4})"}}
                                                        ]
                }
            ]
#add patterns to ruler
ruler.add_patterns(patterns)


#create the doc
doc = nlp(text)

#extract entities
for ent in doc.ents:
    print (ent.text, ent.label_)

## Working with Multi-Word Token Entities and RegEx in spaCy 3


In [32]:
import re

text = "Paul Newman was an American actor, but Paul Hollywood is a British TV Host. The name Paul is quite common."

pattern = r"Paul [A-Z]\w+"

matches = re.finditer(pattern, text)

for match in matches:
    print (match)

<re.Match object; span=(0, 11), match='Paul Newman'>
<re.Match object; span=(39, 53), match='Paul Hollywood'>


In [33]:
import re
import spacy
from spacy.tokens import Span

In [34]:
text = "Paul Newman was an American actor, but Paul Hollywood is a British TV Host. The name Paul is quite common."
pattern = r"Paul [A-Z]\w+"

In [35]:
nlp = spacy.blank("en")
doc = nlp(text)

In [36]:
original_ents = list(doc.ents)

In [37]:
mwt_ents = []
for match in re.finditer(pattern, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end)
    if span is not None:
        mwt_ents.append((span.start, span.end, span.text))

In [38]:
for ent in mwt_ents:
    start, end, name = ent
    per_ent = Span(doc, start, end, label="PERSON")
    original_ents.append(per_ent)

In [39]:
doc.ents = original_ents

In [40]:
for ent in doc.ents:
    print (ent.text, ent.label_)

Paul Newman PERSON
Paul Hollywood PERSON


### Geben Sie größeren Spannen Vorrang

In [ ]:
import re
import spacy

text = "Paul Newman was an American actor, but Paul Hollywood is a British TV Host."
pattern = r"Hollywood"

nlp = spacy.load("en_core_web_sm")

doc = nlp(text)
for ent in doc.ents:
    print (ent.text, ent.label_)

In [41]:
mwt_ents = []
original_ents = list(doc.ents)
for match in re.finditer(pattern, doc.text):
    print (match)
    start, end = match.span()
    span = doc.char_span(start, end)
    if span is not None:
        mwt_ents.append((span.start, span.end, span.text))
for ent in mwt_ents:
    start, end, name = ent
    per_ent = Span(doc, start, end, label="CINEMA")
    original_ents.append(per_ent)

doc.ents = original_ents

<re.Match object; span=(0, 11), match='Paul Newman'>
<re.Match object; span=(39, 53), match='Paul Hollywood'>


ValueError: ignored

In [42]:
from spacy.util import filter_spans
filtered = filter_spans(original_ents)
doc.ents = filtered
for ent in doc.ents:
    print (ent.text, ent.label_)

Paul Newman PERSON
Paul Hollywood PERSON
